In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit


### Install adk

In [5]:
!pip install --upgrade --quiet google-adk

In [6]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
MODEL_NAME = "gemini-2.5-flash"

print(MODEL_NAME)

gemini-2.5-flash


In [7]:
!gcloud config set project {PROJECT_ID}
!gcloud auth application-default login

Reauthentication required.
Please enter your password:Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=TojfvPOeV2DhA5ib3DL9CaJtoEtmYS&access_type=offline&code_challenge=RIwdGhnLb_Tob_QIv-mAYoGINsmJhZz8RtUyPLTYBDs&code_challenge_method=S256


Credentials saved to file: [/Users/hangsik/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "ai-hangsik" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


## Agent on adk

In [8]:
#@title Build a Agent with adk.
from google.adk.agents import Agent
from google.adk.tools import google_search

agent = Agent(
    name="search_assistant",
    model=MODEL_NAME,
    instruction="You are a helpful assistant. Answer user questions using Google Search when needed.",
    description="An assistant that can search the web.",
    tools=[google_search] # Use Google search tool
)


/Users/hangsik/Documents/project/adk_workshop/.venv/lib/python3.12/site-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [9]:
#@title Integrate an agent with Agent Engine

from vertexai.preview.reasoning_engines import AdkApp

app = AdkApp(agent=agent)

response = app.stream_query(
   user_id="shins",  # Required
   message="What is the exchange rate from US dollars to Korean currency today ?",
)

for event in response:
  print(event['content']['parts'][0]['text'])


/Users/hangsik/Documents/project/adk_workshop/.venv/lib/python3.12/site-packages/vertexai/preview/reasoning_engines/templates/adk.py:721: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self._tmpl_attrs["credential_service"] = InMemoryCredentialService()
/Users/hangsik/Documents/project/adk_workshop/.venv/lib/python3.12/site-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()


As of November 1, 2025, the exchange rate from US dollars to Korean Won is approximately 1,427.3380 KRW for 1 USD. Other recent figures from October 31, 2025, indicated the rate was around 1,429.2200 KRW for 1 USD. The South Korean Won has been trading around 1,430 per dollar recently, near a six-month low.


## Deploy agent on Agent Engine for a managed service.

In [10]:
#@title Create a bucket to store agent engine artifacts

BUCKET_URI = f"gs://agent-0417"
# !gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

In [ ]:
# # @title Service account
# shell_output = ! gcloud projects describe  $PROJECT_ID
# project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
# SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
# print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

SERVICE_ACCOUNT: 721521243942-compute@developer.gserviceaccount.com


In [ ]:
# !gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
# !gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

In [11]:
#@title Initialize Vertex AI with Staging Bucket.
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

In [12]:
# @title Deploy your agent on Vertex AI

from vertexai import agent_engines

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.ReasoningEngine#vertexai_preview_reasoning_engines_ReasoningEngine_create

remote_agent = agent_engines.create(
    app,
    display_name="currency agent-20251101",
    gcs_dir_name = "ai-agent-adk",
    description="This is a simple agent with adk on agent engine.",
    requirements=[
        "google-adk",
        "cloudpickle==3.0",
    ],
    extra_packages = []
)

Identified the following requirements: {'pydantic': '2.12.3', 'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.124.0'}
The following requirements are missing: {'pydantic', 'google-cloud-aiplatform'}
The following requirements are incompatible: {'cloudpickle==3.1.1 (required: ==3.0)'}
The following requirements are appended: {'pydantic==2.12.3'}
The final list of requirements: ['google-adk', 'cloudpickle==3.0', 'pydantic==2.12.3']
Using bucket agent-0417
Wrote to gs://agent-0417/ai-agent-adk/agent_engine.pkl
Writing to gs://agent-0417/ai-agent-adk/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://agent-0417/ai-agent-adk/dependencies.tar.gz
Bidi stream API mode is not supported yet in Vertex SDK, please use the GenAI SDK instead. Skipping method bidi_stream_query.
Creating AgentEngine
Create AgentEngine backing LRO: projects/721521243942/locations/us-central1/reasoningEngines/1618186446970028032/operations/1630544631248715776
View progress and logs at htt

In [14]:
# @title Query from remote engine.

for event in remote_agent.stream_query(
    user_id="shins",
    message="What is the exchange rate from US dollars to Korean Won currency on today?",
):
  outcome = event['content']['parts'][0]['text']
  print(outcome)

On November 1, 2025, the exchange rate for US dollars to Korean Won is approximately 1 USD to 1427-1429 KRW.

Specifically:
*   Wise reports a mid-market exchange rate of 1 USD to 1,429 KRW as of November 1, 2025.
*   Xe indicates that 1 USD is equal to 1,427.9585 South Korean Won, last updated on November 1, 2025, at 03:53 UTC.
*   Revolut shows an exchange rate of 1 USD = 1,427.3380 KRW.
*   Trading Economics reported the USD/KRW exchange rate at 1,429.2200 on October 31, 2025.

These rates can fluctuate throughout the day and may vary slightly between different financial institutions and currency exchange services.


In [22]:
# @title Helper function to manage reasoning engine.

from vertexai import agent_engines

# Properties of ReasoningEngine class.
# https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/manage/overview

#----------------------------------------
def get_agent_engine(display_name:str):
  try:
    for agent in agent_engines.list():
      if agent.display_name == display_name:
        return agent_engines.get(agent.name)

      else:
        print("No such reasoning engine or Invalid display name.")

  except Exception as e:
    print(e)

#----------------------------------------
def show_agents():
  """
  List reasoning engines.
  """

  try:
    if not agent_engines.list():
      print("No reasoning engines")

    for idx, agent in enumerate(agent_engines.list()):
        print(f"Agent {idx}: \n\tDisplay Name [{agent.display_name}] \n\tName [{agent.name}] \n\tCreation Time [{agent.create_time}] \n\tResource Name [{agent.resource_name}]\n")

  except Exception as e:
    print(e)

#----------------------------------------

def delete_agent(name):
  """
  Delete a reasoning engines.
  @param name: The name of the reasoning engine.
  @type name: str
  """

  try:
    re = agent_engines.get(name)
    re.delete()
    print(f"Deleted {name}")
  except Exception as e:
    print(e)


## Agent Management

In [23]:
#@title Show agent list
show_agents()

Agent 0: 
	Display Name [currency agent-20251101] 
	Name [1618186446970028032] 
	Creation Time [2025-11-01 13:55:15.986889+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/1618186446970028032]

Agent 1: 
	Display Name [agent_us_central1-2] 
	Name [5875213994741989376] 
	Creation Time [2025-11-01 12:17:44.735774+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/5875213994741989376]

Agent 2: 
	Display Name [agent_us_central1] 
	Name [3146032620555468800] 
	Creation Time [2025-11-01 11:43:06.868856+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/3146032620555468800]

Agent 3: 
	Display Name [agent_us_central1] 
	Name [5561087920732897280] 
	Creation Time [2025-11-01 11:43:17.196871+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/5561087920732897280]

Agent 4: 
	Display Name [agent_us_central1] 
	Name [7428955866184810496] 
	Creation Time [2025-1

In [24]:
#@title Get a agent with display name.
agent = get_agent_engine(display_name="currency agent-20251101")
agent

resource name: projects/ai-hangsik/locations/us-central1/reasoningEngines/1618186446970028032

In [26]:
# @title Query from remote engine.
for event in agent.stream_query(
    user_id="shins",
    message="What is the exchange rate from US dollars to Korean Won currency on toady?",
):
  outcome = event['content']['parts'][0]['text']
  print(outcome)

As of November 1, 2025, the exchange rate from US dollars (USD) to Korean Won (KRW) is approximately 1 USD to 1429.22 KRW. On October 31, 2025, the USD/KRW exchange rate fell to 1429.2200.

The bid price for USD/KRW is around 1428.91 and the ask price is around 1429.53. Other sources indicate rates such as 1 USD = 1427.3380 KRW.


In [27]:
#@title Delete a agent with name.
delete_agent(name="3146032620555468800")

400 The ReasoningEngine "projects/ai-hangsik/locations/us-central1/reasoningEngines/3146032620555468800" contains child resources: sessions. Please delete the child resources before deleting the ReasoningEngine, or set force to true to delete child resources.


## End of notebook